### 분석 목적

온라인 샵 유저의 이동 경로를 파악해, 구매 병목 현상이 일어나는 지점을 발견하고자 함.

### 분석 과정


<b>네트워크 알고리즘</b>을 활용해 유저의 이동 경로를 시각화 하여 파악.

GA를 통해 유의미하다고 판단 된 척도로 소비자를 그룹핑해(6 group) 네트워크 분석 시행.  

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pickle
from collections import Counter
import numpy as np
import pyodbc
import pandas.io.sql as pdsql
import pandas as pd
import re 

pd.set_option('max_columns', 10)

In [ ]:
with open("c:\\Users\\LG\\Desktop\\ipClassified.txt", 'rb') as f:
    ipClassified = pickle.load(f)  #구매,비구매 고객간 구별 위한 ip pickle file
                                   #('210.105.33', 1) 형태로 0,1로 구매,비구매 구분

#비구매 iplist 추출        
NotBuy_IpList = [logip for (logip, _bool) in ipClassified if _bool == 0]

query_logip = \
"""' or logip = '""".join(NotBuy_IpList)

query = \
"""
select before_url
logip, logdate, before_url, after_url
from LogIntake
where logip = '{}'
""".format(query_logip)

print("logip: {}".format(NotBuy_IpList[100]))

In [ ]:
#DB 접속
conn = pyodbc.connect(driver='{SQL Server}', server='175.114.47.85', UID='LYJ_admin', PWD='dudwns06', database='intake')
df = pdsql.read_sql_query(query, con=conn)
conn.close()

# 상위 1000개 ip의 로그만 추출
df = df.head(1000)

In [ ]:
#log url 전처리(네트워크 알고리즘으로 시각화 위해 url 단축, 라벨링) 
#before_url과 after_url은 유저의 방문 기록을 나타내는 변수

df['before_url'] = df['before_url'].apply(lambda x: re.sub('https://www.shopintake.com[\/]*', '', x))
df['after_url'] = df['after_url'].apply(lambda x: re.sub('https://www.shopintake.com[\/]*', '', x))

df['before_url'] = df['before_url'].apply(lambda x: re.sub('\/app\/auth\/login\/\?token=(.*)url=', '', x))
df['after_url'] = df['after_url'].apply(lambda x: re.sub('\/app\/auth\/login\/\?token=(.*)url=', '', x))

df['before_url'] = df['before_url'].apply(lambda x: re.sub('\/app\/notice\/\?os_type=ios', '', x))
df['after_url'] = df['after_url'].apply(lambda x: re.sub('\/app\/notice\/\?os_type=ios', '', x))

df['before_url'] = df['before_url'].apply(lambda x: re.sub('\?(.*)', '', x))
df['after_url'] = df['after_url'].apply(lambda x: re.sub('\?(.*)', '', x))

df['before_url'] = df['before_url'].apply(lambda x: re.sub('static(.*)', '', x))
df['after_url'] = df['after_url'].apply(lambda x: re.sub('static(.*)', '', x))

df = df[df['before_url'] != '']
df = df[df['after_url'] != '']

In [ ]:
# Group 전처리
df['before_url'] = df['before_url'].apply(lambda x: re.sub("view\/[0-9]*/", "view/number/", x))
df['after_url'] = df['after_url'].apply(lambda x: re.sub("view\/[0-9]*/", "view/number/", x))

df['before_url'] = df['before_url'].apply(lambda x: re.sub("category\/[A-Za-z_]*/", "category/subcategory/", x))
df['after_url'] = df['after_url'].apply(lambda x: re.sub("category\/[A-Za-z_]*/", "category/subcategory/", x))

In [ ]:
beforeUrlList = df['before_url'].tolist()
afterUrlList = df['after_url'].tolist()

# 전처리 된 url 간 발생한 모든 접속 경로를 표현하기 위해 edge와 node 생성. 
# 접속 경로는 (before_url,after_url) 형태로 짝지어 표현한다.
nodeList = list(set(beforeUrlList + afterUrlList))
edgeList = list(zip(beforeUrlList, afterUrlList))

In [ ]:
#네트워크 분석

nodes = []
edges = []

G = nx.DiGraph()

G.add_nodes_from(nodeList)
G.add_edges_from(edgeList)

# edge link로 필터링
edgeCount = Counter(edgeList)
remove_by_edge = [(before, after) for (before, after) in edgeCount.keys() if edgeCount[(before, after)] < 10]
G.remove_edges_from(remove_by_edge)

# degree로 필터링
remove_by_degree = [node for node, degree in nx.degree(G) if degree == 0]
G.remove_nodes_from(remove_by_degree)

pos = nx.spring_layout(G)

nx.draw_networkx(G, pos)

plt.show()